# Fase 2: Limpieza y Transformación de Datos

Objetivo: Corregir los problemas identificados en la Fase 1 y enriquecer el dataset para prepararlo para el modelado. La calidad y el rendimiento de tu futuro modelo dependen casi por completo de la calidad de esta fase.



### Manejo de nulos
* **Código de Ejemplo:** `df['col'].fillna(df['col'].median())` o `df.dropna()`
* **Consejo Profesional:** Usa la mediana si hay valores atípicos (outliers). Eliminar filas o columnas es el último recurso.

---

### Conversión de tipos
* **Código de Ejemplo:** `df['fecha'] = pd.to_datetime(df['fecha'])`
* **Consejo Profesional:** Los identificadores (IDs) numéricos deberían tratarse como texto (strings) para evitar cálculos incorrectos.

---

### Eliminar duplicados
* **Código de Ejemplo:** `df.drop_duplicates(inplace=True)`
* **Consejo Profesional:** Revisa los duplicados antes de eliminarlos, ya que su presencia puede indicar errores en el proceso de recolección de datos.

---

### Detección y tratamiento de outliers
* **Código de Ejemplo:**
    ```python
    Q1 = df['col'].quantile(0.25)
    Q3 = df['col'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['col'] < Q3 + 1.5*IQR]
    ```
* **Consejo Profesional:** Visualiza los datos con un `boxplot` de Seaborn (`sns.boxplot`). Otras técnicas incluyen la transformación logarítmica o la winsorización.

---

### Ingeniería de variables
* **Código de Ejemplo:** `df['volumen_total'] = df['sets'] * df['reps']` o `df['dia_semana'] = df['fecha'].dt.dayofweek`
* **Consejo Profesional:** En este paso es donde aplicas tu conocimiento del negocio para crear nuevas características relevantes.

---

### Codificar categóricas
* **Código de Ejemplo:** `pd.get_dummies(df, columns=['cat1'], drop_first=True)`
* **Consejo Profesional:** Usa `drop_first=True` para evitar la multicolinealidad. Para variables con un orden inherente (ordinales), considera usar `OrdinalEncoder`.

---

### Escalado de datos
* **Código de Ejemplo:**
    ```python
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    ```
* **Consejo Profesional:** También puedes considerar otras técnicas como `MinMaxScaler` (para escalar a un rango específico) o `RobustScaler` (si hay outliers).

---

### Guardar datos procesados
* **Código de Ejemplo:** `df.to_csv('data/processed/datos_limpios.csv', index=False)`
* **Consejo Profesional:** Usa `index=False` para evitar que el índice del DataFrame se guarde como una columna en el archivo CSV.

---


## [__] verificar nombres de columnas 
## [__] Identificar columnas con un solo valor
## [__] datos a minuscula

---


## [__] 1. Eliminar duplicados


In [ ]:
# [   ]Calcula el número de duplicados antes de eliminarlos.
dups_before = df_clean.duplicated().sum()

#[    ] Elimina las filas duplicadas, manteniendo la primera aparición.
df_clean.drop_duplicates(inplace=True)

#Consejo Profesional: A veces, las filas pueden estar duplicadas
# solo en un subconjunto de columnas clave (ej., id_cliente y 
# fecha_compra). Puedes especificar estas columnas con el parámetro
# subset: df_clean.drop_duplicates(subset=['id_cliente', 
# 'fecha_compra'], inplace=True).



## [__] Revisar duplicados parciales : mismo nombre y correo pero diferente ID
## [__] Revisar claves primarias  ¿hay valores repetidos

---

## [__] 2. Eliminar o imputar valores nulos


In [ ]:
# [   ] --- Estrategia 1: Imputación para columnas numéricas ---
# Si la distribución es sesgada (como viste en el histograma), la mediana es más robusta que la media.
median_value = df_clean['columna_numerica_con_nulos'].median()
df_clean['columna_numerica_con_nulos'].fillna(median_value, inplace=True)

# [   ] --- Estrategia 2: Imputación para columnas categóricas ---
# La moda (el valor más frecuente) es la mejor opción para imputar categorías.
mode_value = df_clean['columna_categorica_con_nulos'].mode()[0]
df_clean['columna_categorica_con_nulos'].fillna(mode_value, inplace=True)

# [   ] --- Estrategia 3: Eliminación de filas ---
# Úsalo como último recurso si una fila tiene demasiados datos importantes faltantes.
# 'subset' especifica la columna a revisar para la eliminación.
df_clean.dropna(subset=['columna_clave_con_nulos'], inplace=True)

#[  ]Consejo Profesional: La decisión de imputar o eliminar 
# depende del contexto del negocio y la cantidad de datos 
# faltantes. Si una columna tiene >50% de nulos, considera 
# eliminar la columna por completo.



---

## [__] 3. Corrección de Tipos de Datos (Dtypes)
Meta: Asegurarse de que cada columna tenga el tipo de dato correcto para poder realizar operaciones y análisis adecuados.




In [ ]:
#[____] --- Convertir a tipo Fecha (datetime) ---
# Es fundamental para poder realizar operaciones de series de tiempo.
# 'errors='coerce'' convertirá las fechas no válidas en NaT (Not a Time),
# que puedes manejar después.
df_clean['columna_fecha'] = pd.to_datetime(df_clean['columna_fecha'], errors='coerce')


# [___]--- Convertir a tipo Categórico (category) ---
# Es más eficiente en memoria que el tipo 'object' para columnas con un número 
# limitado de valores únicos.
df_clean['columna_a_categoria'] = df_clean['columna_a_categoria'].astype('category') 


# [___] --- Convertir a tipo Numérico (int, float) ---
# Útil si tienes números almacenados como texto (ej. '$1,200.50').
# Primero, necesitas limpiar los caracteres no numéricos.
df_clean['columna_dinero'] = df_clean['columna_dinero'].replace({'\$': '', ',': ''}, regex=True).astype(float)

### [__] Eliminar simbolos no numericos
### [__] Detectar valores negativos donde no deberia
### [__] elimianr caracteres especiales si es posible


---

# [__] 4. Corregir formatos


In [ ]:
df['col'] = df['col'].str.strip().str.lower().str.replace('  ', ' ')

---

## [___] 5. Manejo de Outliers en Análisis de Datos




### [__] Detección de Outliers
    Métodos:
    --    * Boxplot / Gráficos
       * Z-Score
            ¿Qué es? Calcula cuántas desviaciones estándar se aleja un punto de la media. Un umbral común es un Z-score de +/- 3.
            Ideal para: Datos que siguen una distribución normal (Gaussiana). Es sensible a los propios outliers que inflan la media y la desviación estándar.
        * IQR
                ¿Qué es? Un outlier es cualquier valor que cae fuera del siguiente rango: [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR].
                Ideal para: Distribuciones asimétricas o cuando no quieres asumir una distribución normal. Es el método más común y robusto.
        Distribución y visualización
        MAD (desviación absoluta de la mediana)
            se utiliza para medir la dispersión de un conjunto de datos, especialmente cuando se busca una medida robusta que no sea sensible a valores atípicos o extremos

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#[__] Visualizar
sns.boxplot(data=df[['columna1', 'columna2']])
plt.title('Boxplot para detectar outliers')
plt.show()

# [__] Z-Score
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df.select_dtypes(include='number')))
df_outliers_z = df[(z_scores > 3).any(axis=1)]


#[__] IQR
Q1 = df['columna'].quantile(0.25)
Q3 = df['columna'].quantile(0.75)
IQR = Q3 - Q1
lim_inf = Q1 - 1.5 * IQR
lim_sup = Q3 + 1.5 * IQR
outliers_iqr = df[(df['columna'] < lim_inf) | (df['columna'] > lim_sup)]


### [__] ✂️ 2. Eliminación de Outliers

    Cuándo usar:
        Cuando el outlier es un error de medición evidente.
        Cuando es poco probable que ese valor se repita.
        Cuando distorsiona demasiado las métricas estadísticas.

In [ ]:
# IQR para eliminar
df_filtrado = df[(df['columna'] >= lim_inf) & (df['columna'] <= lim_sup)]
# Z-score para eliminar
df_filtrado = df[(z_scores < 3).all(axis=1)]


✅ Recomendaciones:
*    Documenta cuántos valores se eliminaron.
*    Evalúa el impacto en el tamaño del dataset.
*    Guarda copia del dataset original.

### [___] 🔁 3. Reemplazo (Winsorización)

📌 Objetivo: Sustituir outliers por valores extremos válidos sin perder la fila.

* Qué es? Se "aplanan" los outliers. Cualquier valor por encima del percentil 95 (por ejemplo) se reemplaza por el valor del percentil 95. Lo mismo para el extremo inferior (ej. percentil 5).
* Cuándo usarla? Cuando crees que los outliers son legítimos pero su magnitud extrema está afectando negativamente al modelo (especialmente modelos lineales). Conservas la idea de que es un valor "alto" o "bajo" sin que su escala domine.

Cuándo usar:

    Cuando quieres mantener todos los datos pero reducir la influencia de extremos.

    Cuando el outlier puede ser real pero necesitas controlarlo estadísticamente.

✅ Recomendaciones:

    Winsorizar especialmente útil si vas a usar modelos sensibles (como regresión lineal).

    Anota los límites usados (lim_inf, lim_sup).

In [ ]:
from scipy.stats.mstats import winsorize

# Winsoriza al 5% en cada extremo (deja el 90% de los datos centrales intactos)
df['columna_winsorizada'] = winsorize(df['columna'], limits=[0.05, 0.05])

df['columna'] = np.where(df['columna'] > lim_sup, lim_sup,
                  np.where(df['columna'] < lim_inf, lim_inf, df['columna']))



### [__] 🔄 4. Imputación de valores Mediana o Media

📌 Objetivo: Sustituir outliers por valores estadísticos como la media o mediana.

* ¿Qué es? Se sustituye el valor atípico por la mediana (más robusta) o la media de la columna.
* ¿Cuándo usarla? Cuando no quieres perder los datos de las otras columnas de esa fila. Es un método simple y rápido. Prefiere la mediana sobre la media si la distribución es asimétrica.

Cuándo usar:

    Cuando los outliers son errores pero no puedes eliminar datos.

    Si los datos son escasos o críticos y no quieres perder información

 Recomendaciones:

    Usa la mediana en vez de la media si los datos están sesgados.

    Documenta los valores imputados.

In [ ]:
mediana = df['columna'].median()
df['columna_imputada'] = df['columna'].apply(lambda x: mediana if x < limite_inferior or x > limite_superior else x)


### [__] 🔁 5. Transformaciones de datos

Aplicas una función matemática a toda la variable para reducir el impacto de los outliers.

¿Qué es? Funciones como la logarítmica, raíz cuadrada o Box-Cox comprimen la escala de la variable, acercando los outliers al resto de los datos.

¿Cuándo usarla?

        Cuando la variable tiene una fuerte asimetría positiva (una cola larga a la derecha).

        En modelos que asumen normalidad o linealidad (como la regresión lineal).

        Cuando los outliers son una característica intrínseca de la distribución (ej. ingresos, tamaños de empresas).

✅ Recomendaciones:

    Usa log1p si hay ceros.

    Visualiza después de transformar (histplot, boxplot).

    No olvides destransformar al interpretar resultados.

In [ ]:
# Logaritmo
df['columna_log'] = np.log1p(df['columna'])

# Raíz cuadrada
df['columna_sqrt'] = np.sqrt(df['columna'])

# Box-Cox (requiere valores > 0)
from scipy.stats import boxcox
df['columna_boxcox'], _ = boxcox(df['columna'] + 1)


### [__] 🛡️ 6. Uso de Modelos Robustecidos
📌 Objetivo: Modelar sin eliminar ni transformar outliers directamente.
💡 Cuándo usar:

    Cuando los outliers son inevitables.

    Cuando usas modelos resistentes como árboles o medianas.

💻 Modelos comunes:

    RandomForest, XGBoost, GradientBoosting

    HuberRegressor, RANSACRegressor (de sklearn.linear_model)

    IsolationForest (para detección automática)

Modelos Robustos:

    Árboles de Decisión y ensambles (Random Forest, Gradient Boosting): Son naturalmente robustos a los outliers porque dividen el espacio de características en regiones y no se ven afectados por la magnitud de los valores.

    Regresiones Robustas (ej. RANSAC, Huber Regressor): Modelos lineales que ponderan menos los errores grandes, reduciendo la influencia de los outliers.

    Recomendación: Si eliges esta vía, simplemente alimenta los datos originales (o con mínima limpieza) a uno de estos modelos

### [__] 7. Análisis Individual o Validación del Outlier
📌 Objetivo: Determinar si un outlier es realmente un error o un caso especial.
💡 Cuándo usar:

    Cuando un valor parece raro pero puede ser legítimo.

    En áreas sensibles como medicina, fraude, accidentes, etc.

✅ Recomendaciones:

    Revisa el caso completo: ¿otros campos están bien?

    Consulta con el dominio experto si es posible.

    Nunca elimines automáticamente sin investigar.

### [___] 8. Documentación del tratamiento

    🔎 ¿Cómo se detectaron?

    🔁 ¿Qué método se aplicó?

    📉 ¿Cuántos valores se afectaron?

    🧾 ¿Qué decisiones se tomaron?

---
## [___] 6. Crear variables útiles (feature engineering)

In [ ]:
df['duracion_total'] = df['dias'] * df['horas_dia']

| Técnica                          | Descripción breve                                            | Ejemplo de código                                                       | ¿Cuándo usar?                                             |
| -------------------------------- | ------------------------------------------------------------ | ----------------------------------------------------------------------- | --------------------------------------------------------- |
| 🔢 Transformación matemática     | Aplica log, raíz, potencias para normalizar o ajustar escala | `df['log_precio'] = np.log1p(df['precio'])`                             | Cuando hay sesgo o colas largas                           |
| ➗ Variables combinadas/derivadas | Combina columnas para crear relaciones útiles                | `df['precio_m2'] = df['precio'] / df['area']`                           | Cuando dos columnas están relacionadas                    |
| 🔁 Encoding categórico           | Convierte texto a números                                    | `pd.get_dummies(df, columns=['genero'])`                                | Al usar modelos que requieren datos numéricos             |
| 📆 Variables de fecha            | Extrae partes de una fecha o calcula duración                | `df['mes'] = df['fecha'].dt.month`                                      | Si tienes fechas (ventas, eventos, registros, etc.)       |
| 🔢 Frecuencia o conteo           | Crea una nueva variable con la frecuencia de ocurrencia      | `df['freq'] = df['ciudad'].map(df['ciudad'].value_counts())`            | Para capturar importancia o rareza de categorías          |
| 🧱 Binning / Discretización      | Convierte valores continuos a rangos                         | `pd.cut(df['edad'], bins=[0,18,35,60], labels=[...])`                   | Para segmentar poblaciones o estabilizar valores extremos |
| ✅ Indicadores (flags)            | Crea columnas booleanas según condiciones                    | `df['es_vip'] = df['gasto'] > 100000`                                   | Para marcar eventos o atributos clave                     |
| 📊 Estadísticas por grupo        | Calcula medias, desvíos por grupo                            | `df['media_ciudad'] = df.groupby('ciudad')['precio'].transform('mean')` | Para contextualizar los datos dentro de grupos            |
| ✖️ Interacciones entre variables | Multiplica o combina variables para capturar relaciones      | `df['edad_x_ingresos'] = df['edad'] * df['ingresos']`                   | Cuando sospechas relaciones no lineales o sinérgicas      |


----
## [___] 7. Escalar datos (si es necesario)

| Escenario                                                    | ¿Escalar?                                           | ¿Qué tipo?                      |
| ------------------------------------------------------------ | --------------------------------------------------- | ------------------------------- |
| Árboles (Decision Tree, Random Forest, XGBoost)              | ❌ No necesario                                      | Ninguno                         |
| Regresiones lineales / logísticas                            | ✅ Sí                                                | Estandarización o normalización |
| Modelos de ML sensibles a distancia (KNN, SVM, PCA, K-means) | ✅ Sí                                                | Recomendado                     |
| Redes neuronales (MLP, Deep Learning)                        | ✅ Imprescindible                                    | Normalización (0-1)             |
| Datos categóricos codificados (one-hot)                      | ❌ No                                                | -                               |
| Datos con outliers                                           | ⚠️ Escalar con RobustScaler o usar métodos robustos | Sí                              |

----------------------------------------------

| Escalador          | ¿Cómo funciona?                            | Cuándo usarlo                  | Código                             |
| ------------------ | ------------------------------------------ | ------------------------------ | ---------------------------------- |
| **StandardScaler** | Media = 0, desviación estándar = 1         | Datos normalmente distribuidos | `StandardScaler().fit_transform()` |
| **MinMaxScaler**   | Escala entre 0 y 1                         | Redes neuronales o imágenes    | `MinMaxScaler().fit_transform()`   |
| **RobustScaler**   | Usa mediana y IQR (no sensible a outliers) | Datos con muchos outliers      | `RobustScaler().fit_transform()`   |
| **Normalizer**     | Normaliza filas (norma L2 = 1)             | Series temporales, clustering  | `Normalizer().fit_transform()`     |


🎛️ Tipos de escalado (los más usados)
### [__] 1. StandardScaler (Estandarización)

    Lo que hace: centra los datos en media cero y los escala con varianza uno. Esto quiere decir que convierte tus datos a una distribución estándar (z-score).

    Fórmula:
    z=x−μσ
    z=σx−μ​

    donde μ es la media y σ la desviación estándar.

    Cuándo usarlo:

        Cuando tus datos están cercanos a una distribución normal (campana).

        En modelos lineales (regresión lineal, logística), PCA, SVM, redes neuronales.

    Cuidado: no es robusto frente a outliers. Los valores extremos distorsionan la media y σ.

    🧠 Consejo: Antes de usarlo, puedes graficar un histograma o sns.kdeplot para verificar la forma de la distribución.
    

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Asumiendo que 'X' son tus características y 'y' tu objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

# Ajusta el escalador SOLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transforma los datos de prueba con el escalador ya ajustado
X_test_scaled = scaler.transform(X_test)

# El resultado es un array de NumPy, puedes convertirlo de nuevo a DataFrame si lo deseas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

### [__]  2. MinMaxScaler (Normalización)

    Lo que hace: escala todos los valores en un rango definido, usualmente entre 0 y 1.

    Fórmula:
    x′=x−min⁡(x)max⁡(x)−min⁡(x)
    x′=max(x)−min(x)x−min(x)​

    Cuándo usarlo:

        En modelos basados en gradientes como redes neuronales (MLP, CNN, etc.).

        En datos de imágenes (por ejemplo, pixel values de 0 a 255 → escalar a 0-1).

        Cuando necesitas mantener la forma de la distribución, pero limitar el rango.

    Cuidado: es muy sensible a outliers porque usa los valores extremos para definir los límites del rango.

    🧠 Consejo: Si usas MinMaxScaler, asegúrate de haber tratado outliers antes (con IQR, winsorización, etc.).
    

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1)) # El rango es personalizable

# Ajusta y transforma de la misma manera que StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### [__]  3. RobustScaler

    Lo que hace: utiliza la mediana y el rango intercuartílico (IQR) en lugar de la media y desviación estándar. Esto hace que sea resistente a valores extremos.

    Fórmula:
    x′=x−medianaIQR
    x′=IQRx−mediana​

    Cuándo usarlo:

        Cuando sabes que tu dataset tiene outliers fuertes que no deseas eliminar.

        En análisis financiero, donde los datos suelen tener colas largas o picos de valores.

    🧠 Consejo: Muy útil como paso previo a modelos de ML donde no puedes permitir que los outliers distorsionen tu resultado, pero no puedes o no quieres eliminarlos.



In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

# Ajusta y transforma
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### [__]  4. Normalizer

    Lo que hace: normaliza cada fila del dataset en lugar de cada columna. Se usa la norma L1 o L2 para hacer que el vector tenga una longitud de 1.

    Ejemplo: útil para datos donde cada fila representa un vector, como en texto (TF-IDF), series temporales o clustering de comportamiento.

    Cuándo usarlo:

        En algoritmos que dependen de la dirección del vector más que de su magnitud, como clustering, técnicas de texto, etc.

    ⚠️ Consejo: No confundas Normalizer con MinMaxScaler. Uno normaliza por fila, otro por columna.

Escala cada fila (es decir, cada muestra) para que su norma (longitud del vector) sea igual a 1.

Se usa en algoritmos que trabajan con dirección del vector más que con su magnitud:

    Similaridad de texto (TF-IDF, Word embeddings)

    Clustering (KMeans)

    Datos temporales o de sensores

In [ ]:
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np

# Supongamos un DataFrame con 3 características por muestra
data = np.array([
    [3.0, 4.0, 0.0],
    [1.0, 2.0, 2.0],
    [0.0, 0.0, 10.0]
])

df = pd.DataFrame(data, columns=['feature1', 'feature2', 'feature3'])

# Inicializar el normalizador con norma L2 (por defecto)
normalizer = Normalizer(norm='l2')  # También puedes usar 'l1' o 'max'

# Aplicar normalización por fila
data_normalized = normalizer.fit_transform(df)

# Convertir a DataFrame para ver resultados
df_normalizado = pd.DataFrame(data_normalized, columns=df.columns)

print("Original:")
print(df)
print("\nNormalizado (L2):")
print(df_normalizado)


¿Cómo escalar correctamente un dataset?

    Trata los NaN antes. El escalador no puede trabajar con valores faltantes.

    Separa tu dataset en entrenamiento y prueba antes de escalar.

    Ajusta (fit) el escalador solo con el set de entrenamiento.

    Transforma (transform) tanto entrenamiento como prueba con ese mismo escalador.

    Guarda el escalador (si el modelo va a producción).

🚫 Errores comunes

    Escalar después de entrenar el modelo (esto genera data leakage).

    Escalar sin eliminar o tratar outliers (afecta a StandardScaler y MinMaxScaler).

    Escalar variables categóricas convertidas a dummies (one-hot), cuando no hace falta.

    Aplicar fit_transform() al set de prueba. ¡Solo se usa transform()!

🧠 Consejos prácticos

    En proyectos de IA o deep learning, usa MinMaxScaler o normaliza a [-1, 1] para que la red converja mejor.

    En modelos financieros o con valores dispersos, prueba primero con RobustScaler.

    Si vas a usar PCA o clustering, escalado previo es obligatorio para que no domine una variable.

    Si no sabes qué usar, prueba con StandardScaler y compara resultados.

    Guarda tu escalador con joblib.dump(scaler, 'nombre_scaler.pkl') para reproducibilidad.

| Punto                             | Estandarización            | Normalización                                                 |
| --------------------------------- | -------------------------- | ------------------------------------------------------------- |
| Centrado en media                 | ✅ Sí                       | ❌ No                                                          |
| Rango controlado (ej: 0 a 1)      | ❌ No                       | ✅ Sí                                                          |
| Forma de distribución se mantiene | ❌ No (transforma la forma) | ✅ Sí (solo cambia el rango)                                   |
| Robusto contra outliers           | ❌ No                       | ❌ No (ambos son sensibles, usar RobustScaler si hay outliers) |
| Escenarios típicos                | PCA, SVM, regresión        | Deep Learning, imágenes, magnitudes distintas                 |
| Afecta unidades                   | Sí                         | Sí                                                            |


---
## [___] 8. Guardar archivo limpio

In [ ]:
df.to_csv("data/processed/dataset_limpio.csv", index=False)
